In [238]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import cv2
import nibabel as nib
from scipy import ndimage
import tensorflow as tf
import scipy
from scipy.stats import wilcoxon
from scipy.ndimage.measurements import label
from matplotlib.collections import LineCollection

In [2]:
posPatients = np.concatenate((np.array([2,4,5,7,9,11,12,13,16,17,18,19,21,22]),range(23,52),np.array([55]),
                                range(60,63),range(64,74),range(75,82),range(83,104),range(105,109)))
locList = []
maskList = []
posList = []
img_height=128
limit=0.25
pixLimit=5

for i in range(len(posPatients)):
    if posPatients[i]<23:
        if posPatients[i]==12:
            img_path = 'C:/Users/Oona/Documents/Tpc/hnc/anom_data/positiiviset/12_anon/12_maski/12_maskit.img'
        else:
            img_path = 'C:/Users/Oona/Documents/Tpc/hnc/anom_data/positiiviset/{}_anon/{}_maski/{}_maski.img'.format(
                        posPatients[i],posPatients[i],posPatients[i])
    else:
        if posPatients[i]==23:
            img_path = 'C:/Users/Oona/Documents/Tpc/hnc/anom_data/positiiviset/23/23_maski/23_nifti_maski_edit.img'
        if posPatients[i]==24:
            img_path = 'C:/Users/Oona/Documents/Tpc/hnc/anom_data/positiiviset/24/24_maski/24_maski.img'
        if posPatients[i]==38:
            img_path = 'C:/Users/Oona/Documents/Tpc/hnc/anom_data/positiiviset/38/38_nifti_maski/38_nifti_maski_edit.img'
        if posPatients[i] not in [23,24,38]:    
            img_path = 'C:/Users/Oona/Documents/Tpc/hnc/anom_data/positiiviset/{}/{}_nifti_maski/{}_nifti_maski.img'.format(
                        posPatients[i],posPatients[i],posPatients[i])
    img = nib.load(img_path)
    img_data = img.get_fdata()
    locations = []
    masks = []
    for j in range(img_data.shape[2]):
        slice = np.array(img_data[:,:,j])
        if np.max(slice)>0:
            slice=cv2.resize(slice,(img_height,img_height))
            for k in range(img_height):
                for l in range(img_height):
                    if slice[k,l]>limit:
                        slice[k,l]=1
                    else:
                        slice[k,l]=0
            if np.sum(slice)>pixLimit:
                locations.append(j)
                masks.append(slice)
    locList.append(np.array(locations))
    maskList.append(masks)

for i in range(len(posPatients)):
    if posPatients[i]<23:
        img_path = 'C:/Users/Oona/Documents/Tpc/hnc/anom_data/positiiviset/{}_anon/{}_nifti_pet/{}_nifti_pet.img'.format(
                        posPatients[i],posPatients[i],posPatients[i])
    else:
        img_path = 'C:/Users/Oona/Documents/Tpc/hnc/anom_data/positiiviset/{}/{}_nifti_pet/{}_nifti_pet.img'.format(
                        posPatients[i],posPatients[i],posPatients[i])
    img = nib.load(img_path)
    img_data = img.get_fdata()
    pos = []
    for j in range(len(locList[i])):
        pos.append(cv2.resize(np.array(img_data[:,:,locList[i][j]]),(img_height,img_height)))
    posList.append(pos)

for i in range(len(posList)):
    for j in range(len(posList[i])):
        posList[i][j]=(posList[i][j]-np.min(posList[i][j]))/(np.max(posList[i][j])-np.min(posList[i][j]))

In [3]:
numberOfSlices=[]
finPatients=0
for i in range(len(posList)):
    numberOfSlices.append(len(posList[i]))
    if len(posList[i])>0:
        finPatients+=1
numberOfSlices=np.array(numberOfSlices)
print(finPatients)
print(sum(numberOfSlices))
print(np.mean(numberOfSlices))
print(np.std(numberOfSlices))
print(numberOfSlices)

89
962
10.808988764044944
6.063107084385669
[ 3  9 11 14  8  9 15 12  5 10 13  8 11 10  4 13  4  4  6  4  6  6  5 12
 12  6 18 18  6 19  7 12 13 22 10  5 20  5 12 25  5  7  2  9  6 18 24 11
 11 13 21  7  6  9 13  5  2  1  7  3 18 15  6 20 26 16 13 10 11  6  3  4
 15  6 26  6 10 10 20  5 17 13 18 13 14 11  4 11 23]


In [174]:
#create lists of images with one-click pixels

comList=[]
pixList=[]
np.random.seed(131)
structure=np.ones((3,3),dtype=int)

for i in range(len(posPatients)):
    comImgs=[]
    for j in range(len(maskList[i])):
        img=maskList[i][j]
        labeled, ncomponents = label(img,structure)
        img1=np.zeros((img_height,img_height))
        if ncomponents==1:
            com=ndimage.measurements.center_of_mass(img)
            img1[round(com[0]),round(com[1])]=1
        else:
            for u in range(1,ncomponents+1):
                img0=np.zeros((img_height,img_height))     
                for k in range(img_height):
                    for l in range(img_height):
                        if labeled[k,l]==u:
                            img0[k,l]=1
                com=ndimage.measurements.center_of_mass(img0)
                img1[round(com[0]),round(com[1])]=1
        comImgs.append(img1)
    comList.append(comImgs)

for i in range(len(posPatients)):
    pixImgs=[]
    for j in range(len(maskList[i])):
        img=maskList[i][j]
        labeled, ncomponents = label(img,structure)
        img1=np.zeros((img_height,img_height))
        if ncomponents==1:
            options=[]
            for k in range(img_height):
                for l in range(img_height):
                    if img[k,l]==1:
                        options.append(np.array([k,l]))
            onePix=random.choice(options)
            img1[onePix[0],onePix[1]]=1 
        else:
            for u in range(1,ncomponents+1):
                img0=np.zeros((img_height,img_height))     
                for k in range(img_height):
                    for l in range(img_height):
                        if labeled[k,l]==u:
                            img0[k,l]=1
                options=[]
                for k in range(img_height):
                    for l in range(img_height):
                        if img0[k,l]==1:
                            options.append(np.array([k,l]))
                onePix=random.choice(options)
                img1[onePix[0],onePix[1]]=1 
        pixImgs.append(img1)
    pixList.append(pixImgs)

In [175]:
testSet1=np.concatenate((range(0,80,5),np.array([84,83,85])))
testSet2=np.concatenate((range(1,80,5),np.array([80])))
testSet3=np.concatenate((range(2,80,5),np.array([82,87,88])))
testSet4=np.concatenate((range(3,80,5),np.array([81])))
testSet5=np.concatenate((range(4,80,5),np.array([86])))
testSetIndexes=[testSet1,testSet2,testSet3,testSet4,testSet5]
for j in range(5):
    l=0
    for i in range(len(posList)):
        if i in testSetIndexes[j]:
            l+=len(posList[i])
    print(l)

192
191
192
193
194


In [176]:
def splitIntoSets(posList,maskList,comList,pixList,testSetIndexes,type,k):

    #type is 'com' for center of mass, 'pix' for random positive pixel, and 'no' if neither

    x_train = []
    y_train = []
    x_test = []
    y_test = []

    testSetPatients=testSetIndexes[k]

    for i in range(len(posList)):
        if i in testSetPatients:
            for j in range(len(posList[i])):
                img=posList[i][j]
                if type=='com':
                    x_test.append(np.rollaxis(np.array([img,comList[i][j]]),0,3))
                if type=='pix':
                    x_test.append(np.rollaxis(np.array([img,pixList[i][j]]),0,3))
                if type=='no':
                    x_test.append(img)
                y_test.append(maskList[i][j])   
        else:
            for j in range(len(posList[i])):
                img=posList[i][j]
                if type=='com':
                    x_train.append(np.rollaxis(np.array([img,comList[i][j]]),0,3))
                if type=='pix':
                    x_train.append(np.rollaxis(np.array([img,pixList[i][j]]),0,3))
                if type=='no':
                    x_train.append(img)
                y_train.append(maskList[i][j]) 

    x_train=np.array(x_train)
    y_train=np.array(y_train)
    x_test=np.array(x_test)
    y_test=np.array(y_test)

    return([x_train,y_train,x_test,y_test])

In [177]:
def unetForSegmentation(shape):

    img_height=shape[1]
    if len(shape)==4:
        chl_number=shape[3]
    else:
        chl_number=1

    #U-Net model
    inputs = tf.keras.layers.Input(shape=(img_height,img_height,chl_number))

    #Contraction path
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
    c1 = tf.keras.layers.Dropout(0.1)(c1)
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = tf.keras.layers.Dropout(0.1)(c2)
    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = tf.keras.layers.Dropout(0.2)(c3)
    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

    c4 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = tf.keras.layers.Dropout(0.2)(c4)
    c4 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)

    #Expansive path
    u7 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c4)
    u7 = tf.keras.layers.concatenate([u7, c3])
    c7 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = tf.keras.layers.Dropout(0.1)(c7)
    c7 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = tf.keras.layers.concatenate([u8, c2])
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = tf.keras.layers.Dropout(0.1)(c8)
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = tf.keras.layers.Dropout(0.1)(c9)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

    outputs = tf.keras.layers.Conv2D(1, (1,1), activation='linear')(c9) #sigmoid or linear

    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy')

    return model

In [ ]:
for k in range(5):
    for j in ['no','com','pix']:
        x_train,y_train,x_test,y_test=splitIntoSets(posList,maskList,comList,pixList,testSetIndexes,j,k)
        model=unetForSegmentation(x_test.shape)
        numEpochs=50
        earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', patience=30, restore_best_weights=True)
        history=model.fit(x=x_train,y=y_train,epochs=numEpochs,validation_split=0.3,callbacks=[earlystopping],shuffle=True)
        trainPreds=model.predict(x_train)
        predictions=model.predict(x_test)
        plt.plot(range(len(history.history['loss'])),history.history['loss'],color='blue')
        for i in range(len(trainPreds)):
            np.savetxt('results/{}_{}_{}_trainPreds.txt'.format(j,k,i), trainPreds[i][:,:,0])
        for i in range(len(predictions)):
            np.savetxt('results/{}_{}_{}_predictions.txt'.format(j,k,i), predictions[i][:,:,0])

In [36]:
def countDice(img,mask):

    dice=0
    TP=0
    FN=0
    FP=0
    for i in range(img_height):
        for j in range(img_height):
            if mask[i,j]>0.5 and img[i,j]>0.5:
                TP+=1
            if mask[i,j]>0.5 and img[i,j]<0.5:
                FN+=1
            if mask[i,j]<0.5 and img[i,j]>0.5:
                FP+=1
    if TP+FN+FP>0:
        dice=2*TP/(2*TP+FN+FP)

    return(dice)

def dicesForLimit(predictions,y_test,limit):

    dices=np.zeros((len(y_test)))

    img_height=predictions[0].shape[0]

    for k in range(len(y_test)):
        TP=0
        FN=0
        FP=0
        for i in range(img_height):
            for j in range(img_height):
                if y_test[k,i,j]>0.5 and predictions[k,i,j]>limit:
                    TP+=1
                if y_test[k,i,j]>0.5 and predictions[k,i,j]<limit:
                    FN+=1
                if y_test[k,i,j]<0.5 and predictions[k,i,j]>limit:
                    FP+=1
        if TP+FN+FP>0:
            dices[k]=2*TP/(2*TP+FN+FP)

    return(dices)

def turnIntoBinaryMask(img,lim):

    img_height=img.shape[0]

    binM=np.zeros((img_height,img_height))

    for i in range(img_height):
        for j in range(img_height):
            if img[i][j]>lim:
                binM[i][j]=1
    
    return binM

In [33]:
def removePixels(img,clickImg):

    structure=np.ones((3,3),dtype=int)
    labeled, ncomponents = label(clickImg,structure)
    indexes=[]
    for u in range(ncomponents):
        ind=np.unravel_index(np.argmax(clickImg,axis=None),img.shape)
        indexes.append(ind)
        clickImg[ind[0],ind[1]]=0
    labeled, ncomponents = label(img,structure)
    img1=np.zeros((img_height,img_height))
    for ind in indexes:
        for u in range(1,ncomponents+1):
            if labeled[ind[0],ind[1]]==u:
                for i in range(img_height):
                    for j in range(img_height):
                        if labeled[i,j]==u:
                            img1[i,j]=1

    return(img1)

In [179]:
j='com'
k=4
x_train,y_train,x_test,y_test=splitIntoSets(posList,maskList,comList,pixList,testSetIndexes,j,k)

trainPreds=[]
for i in range(len(y_train)):
    trainPreds.append(np.loadtxt('results/{}_{}_{}_trainPreds.txt'.format(j,k,i)))
trainPreds=np.array(trainPreds)

predictions=[]
for i in range(len(y_test)):
    predictions.append(np.loadtxt('results/{}_{}_{}_predictions.txt'.format(j,k,i)))
predictions=np.array(predictions)

In [184]:
limit=0.13
dices=dicesForLimit(trainPreds,y_train,limit)
print(np.median(dices))

0.5714285714285714


In [185]:
limit=0.13
dices=dicesForLimit(predictions,y_test,limit)
print(np.median(dices))
np.savetxt('{}_{}_dices.txt'.format(j,k),dices)

0.6535205848403232


In [97]:
lim=0.38
j='pix'
x_train,y_train,x_test,y_test=splitIntoSets(posList,maskList,comList,pixList,testSetIndexes,j,k)
dices=[]
for i in range(len(y_test)):
    img=turnIntoBinaryMask(predictions[i],lim)
    structure=np.ones((3,3),dtype=int)
    labeled, ncomponents = label(img,structure)
    if ncomponents>1:
        img=removePixels(img,x_test[i][:,:,1])
    dices.append(countDice(img,y_test[i]))
dices=np.array(dices)
print(np.median(dices))

0.6811868686868687


In [200]:
j='1com'
diceList=[]
for k in range(5):
    dices=np.loadtxt('{}_{}_dices.txt'.format(j,k))
    print(round(np.median(dices),3))
    diceList.append(dices)
dices=np.concatenate((diceList[0],diceList[1],diceList[2],diceList[3],diceList[4]))
print(round(np.median(dices),3))

0.604
0.5
0.568
0.533
0.698
0.593


In [212]:
print(wilcoxon(x=noDices,y=pix1Dices))

WilcoxonResult(statistic=29923.0, pvalue=0.04871656838862777)


In [213]:
def f_test(x,y):

    x=np.array(x)
    y=np.array(y)
    f=np.var(x,ddof=1)/np.var(y,ddof=1)
    dfn=x.size-1
    dfd=y.size-1
    p=2*min(1-scipy.stats.f.cdf(f,dfn,dfd),scipy.stats.f.cdf(f,dfn,dfd))
    return(p)

In [ ]:
diceList1=[noDices,pix1Dices,com1Dices,pixDices,comDices]
fig=plt.figure(figsize=(4,4))
axi=fig.add_axes([0,0,1,1])
bplot=axi.boxplot(diceList1,patch_artist=True,positions=[1,2,3,4,5])
colors=5*['white']
for patch,color in zip(bplot['boxes'],colors):
    patch.set_facecolor(color)
for median in bplot['medians']:
    median.set(color='black',linewidth=1.5)
plt.xticks([1,2,3,4,5],['(1)','(2)','(3)','(4)','(5)'],fontsize=13)
plt.yticks(fontsize=13)
fig=plt.gcf()
plt.show()
#fig.savefig('fig_plt_5',bbox_inches='tight')

In [239]:
def get_all_edges(bool_img):
    """
    Get a list of all edges (where the value changes from True to False) in the 2D boolean image.
    The returned array edges has he dimension (n, 2, 2).
    Edge i connects the pixels edges[i, 0, :] and edges[i, 1, :].
    Note that the indices of a pixel also denote the coordinates of its lower left corner.
    """
    edges = []
    ii, jj = np.nonzero(bool_img)
    for i, j in zip(ii, jj):
        # North
        if j == bool_img.shape[1]-1 or not bool_img[i, j+1]:
            edges.append(np.array([[i, j+1],
                                   [i+1, j+1]]))
        # East
        if i == bool_img.shape[0]-1 or not bool_img[i+1, j]:
            edges.append(np.array([[i+1, j],
                                   [i+1, j+1]]))
        # South
        if j == 0 or not bool_img[i, j-1]:
            edges.append(np.array([[i, j],
                                   [i+1, j]]))
        # West
        if i == 0 or not bool_img[i-1, j]:
            edges.append(np.array([[i, j],
                                   [i, j+1]]))

    if not edges:
        return np.zeros((0, 2, 2))
    else:
        return np.array(edges)


def close_loop_edges(edges):
    """
    Combine thee edges defined by 'get_all_edges' to closed loops around objects.
    If there are multiple disconnected objects a list of closed loops is returned.
    Note that it's expected that all the edges are part of exactly one loop (but not necessarily the same one).
    """

    loop_list = []
    while edges.size != 0:

        loop = [edges[0, 0], edges[0, 1]]  # Start with first edge
        edges = np.delete(edges, 0, axis=0)

        while edges.size != 0:
            # Get next edge (=edge with common node)
            ij = np.nonzero((edges == loop[-1]).all(axis=2))
            if ij[0].size > 0:
                i = ij[0][0]
                j = ij[1][0]
            else:
                loop.append(loop[0])
                # Uncomment to to make the start of the loop invisible when plotting
                # loop.append(loop[1])
                break

            loop.append(edges[i, (j + 1) % 2, :])
            edges = np.delete(edges, i, axis=0)

        loop_list.append(np.array(loop))

    return loop_list


def plot_outlines(bool_img, ax=None, **kwargs):
    if ax is None:
        ax = plt.gca()

    edges = get_all_edges(bool_img=bool_img)
    edges = edges - 0.5  # convert indices to coordinates; TODO adjust according to image extent
    outlines = close_loop_edges(edges=edges)
    cl = LineCollection(outlines, **kwargs)
    ax.add_collection(cl)

In [ ]:
i=73 #6,19,35,73
k=0
j='no'
#x_train,y_train,x_test,y_test=splitIntoSets(posList,maskList,comList,pixList,testSetIndexes,j,k)
print(round(np.loadtxt('{}_{}_dices.txt'.format('no',k))[i],3))
print(round(np.loadtxt('{}_{}_dices.txt'.format('1pix',k))[i],3))
print(round(np.loadtxt('{}_{}_dices.txt'.format('1com',k))[i],3))
print(round(np.loadtxt('{}_{}_dices.txt'.format('pix',k))[i],3))
print(round(np.loadtxt('{}_{}_dices.txt'.format('com',k))[i],3))
array0=cv2.rotate(y_test[i],cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)
img=turnIntoBinaryMask(np.loadtxt('results/{}_{}_{}_predictions.txt'.format(j,k,i)),0.23)
array1=cv2.rotate(img,cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)
j='pix'
img=turnIntoBinaryMask(np.loadtxt('results/{}_{}_{}_predictions.txt'.format(j,k,i)),0.25)
array2=cv2.rotate(img,cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)
j='com'
img=turnIntoBinaryMask(np.loadtxt('results/{}_{}_{}_predictions.txt'.format(j,k,i)),0.27)
array3=cv2.rotate(img,cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)
plt.figure()
plt.imshow(cv2.rotate(x_test[i],cv2.cv2.ROTATE_90_COUNTERCLOCKWISE),cmap='gray')
plot_outlines(array0.T, lw=3, color='gray')
plot_outlines(array1.T, lw=3, color='purple')
plot_outlines(array2.T, lw=3, color='green')
plot_outlines(array3.T, lw=3, color='blue')
plt.savefig('fig3d.png',bbox_inches='tight')